# Evolver Loop 70 Analysis

Goal: validate whether a solvent **distance-to-training** signal is predictive of **leave-one-solvent-out error**, supporting applicability-domain (AD) shrinkage/calibration.

We will:
1. Load single-solvent training data.
2. Build a simple, fast baseline model (Ridge) using kinetics + (Spange+ACS) solvent descriptors.
3. For each held-out solvent (group), compute:
   - nearest-neighbor distance to remaining solvents in descriptor space
   - MSE on that solvent
4. Correlate distance vs error and identify outlier solvents.

This is **not** intended to beat the baseline, just to check if AD has a measurable signal.

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

DATA_PATH = '/home/data'

# Load data
single = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
spange = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
acs = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

solvent_col = 'SOLVENT NAME'
solvents = single[solvent_col].unique().tolist()
print('n rows:', len(single), 'n solvents:', len(solvents))

# descriptor matrix per solvent
desc = pd.concat([
    spange.reindex(solvents).fillna(0.0),
    acs.reindex(solvents).fillna(0.0)
], axis=1)
print('descriptor dim:', desc.shape)

# kinetic features
rt = single['Residence Time'].values.reshape(-1,1)
tc = single['Temperature'].values.reshape(-1,1)
tk = tc + 273.15
invT = 1000.0 / tk
logt = np.log(rt + 1e-6)
inter = invT * logt
kin = np.hstack([rt, tc, invT, logt, inter])

# attach solvent descriptors row-wise
X_desc = desc.reindex(single[solvent_col]).values
X = np.hstack([kin, X_desc])

# known target columns for this dataset
y_cols = ['SM','Product 2','Product 3']
Y = single[y_cols].values

# scale features for ridge
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print('X shape:', X_scaled.shape, 'Y shape:', Y.shape)

n rows: 656 n solvents: 24
descriptor dim: (24, 18)
X shape: (656, 23) Y shape: (656, 3)


In [4]:
from sklearn.metrics import mean_squared_error

def nearest_neighbor_dist(solvent_name, train_solvents, desc_df):
    v = desc_df.loc[solvent_name].values.astype(float)
    M = desc_df.loc[train_solvents].values.astype(float)
    # euclidean distance in raw descriptor space
    d = np.sqrt(((M - v)**2).sum(axis=1))
    return float(d.min()), float(d.mean())

results = []
alpha = 1.0

for s in solvents:
    mask_te = single[solvent_col].values == s
    mask_tr = ~mask_te

    tr_solvents = list(set(single.loc[mask_tr, solvent_col].unique().tolist()))
    d_min, d_mean = nearest_neighbor_dist(s, tr_solvents, desc)

    # Ridge per target
    preds = np.zeros((mask_te.sum(), 3))
    for t in range(3):
        model = Ridge(alpha=alpha, random_state=0)
        model.fit(X_scaled[mask_tr], Y[mask_tr, t])
        preds[:, t] = model.predict(X_scaled[mask_te])

    mse = mean_squared_error(Y[mask_te], preds)
    results.append({'solvent': s, 'n_rows': int(mask_te.sum()), 'mse': mse, 'd_nn_min': d_min, 'd_nn_mean': d_mean})

res = pd.DataFrame(results).sort_values('mse', ascending=False)
res.head(10)

,solvent,n_rows,mse,d_nn_min,d_nn_mean
16,Dihydrolevoglucosenone (Cyrene),18,0.281700,22.917080,32.711936
4,Cyclohexane,34,0.106549,7.863806,29.124853
2,"1,1,1,3,3,3-Hexafluoropropan-2-ol",37,0.104999,11.976880,29.788838
1,"Ethylene Glycol [1,2-Ethanediol]",22,0.055519,7.510199,30.740508
3,2-Methyltetrahydrofuran [2-MeTHF],58,0.044185,11.842809,26.777097
8,Acetonitrile.Acetic Acid,22,0.028686,6.354731,20.636261
13,Decanol,20,0.028449,6.180422,20.228442
10,"2,2,2-Trifluoroethanol",37,0.024773,11.976880,27.583655
12,"DMA [N,N-Dimethylacetamide]",41,0.020878,9.423505,28.961021
5,IPA [Propan-2-ol],5,0.016555,6.354731,19.760278


In [5]:
# Correlation distance vs error
corr_min = res[['mse','d_nn_min']].corr().iloc[0,1]
corr_mean = res[['mse','d_nn_mean']].corr().iloc[0,1]

print('corr(mse, nearest-neighbor distance):', corr_min)
print('corr(mse, mean distance):', corr_mean)

# Show scatter-like summary
print('\nTop 8 high-distance solvents:')
print(res.sort_values('d_nn_min', ascending=False).head(8)[['solvent','mse','d_nn_min','d_nn_mean','n_rows']])

print('\nTop 8 high-error solvents:')
print(res.head(8)[['solvent','mse','d_nn_min','d_nn_mean','n_rows']])

corr(mse, nearest-neighbor distance): 0.7164641251663928
corr(mse, mean distance): 0.19798865731038284

Top 8 high-distance solvents:
                              solvent       mse   d_nn_min  d_nn_mean  n_rows
16    Dihydrolevoglucosenone (Cyrene)  0.281700  22.917080  32.711936      18
6                  Water.Acetonitrile  0.011014  13.542013  52.640954      37
11       Water.2,2,2-Trifluoroethanol  0.006641  13.542013  45.398612      22
2   1,1,1,3,3,3-Hexafluoropropan-2-ol  0.104999  11.976880  29.788838      37
10             2,2,2-Trifluoroethanol  0.024773  11.976880  27.583655      37
3   2-Methyltetrahydrofuran [2-MeTHF]  0.044185  11.842809  26.777097      58
12        DMA [N,N-Dimethylacetamide]  0.020878   9.423505  28.961021      41
7                        Acetonitrile  0.012315   9.423505  28.401594      59

Top 8 high-error solvents:
                              solvent       mse   d_nn_min  d_nn_mean  n_rows
16    Dihydrolevoglucosenone (Cyrene)  0.281700  22.917080